## Defining Common Util Functions

In [1]:
'''
Takes a qubitString as input and returns a matrix 
that represents the qubit in the standard basis.
Ex: |00> = [1,0,0,0]
'''
def getQubitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

'''
Given a bitstring, return the decimal number 
represented by the bitstring
'''
def getDecimalNo(bitString):
    val = 0
    n = len(bitString)
    for i in range(0,n):
        val = val + (2**i)*(int)(bitString[-i-1])
    return val

'''
Given a value of n, returns all possible bit-strings of size n.
This function will return a list of 2^n bit strings
'''
def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

## Common function object. 
This is the base class and all functions Deutsch-Jozsa (balanced, constant), Bernstein-Vazirani(different a and b values) inhert from this class

In [2]:
import numpy as np

class FunctionObject:
    def __init__(self,fx,n):
        self.__fx = fx
        self.__n = n
        self.__Uf = self.__createUf()
    
    '''
    Apply the function on the given input.
    '''
    def applyFx(self,input):
        return self.__fx(input)
        
    '''
    The N value which signifies the no. of qubits.
    '''
    def getN(self):
        return self.__n
    
    '''
    The Uf matrix which represents the oracle for this function
    '''
    def getUf(self):
        return self.__Uf
    
    '''
    Given the function to execute, and the qubits |x1x2..xn> and the ancilla|b>.
    This function will return a bit vector representing |x1x2..xn> |b+f(x)>
    '''
    def getFunctionResult(self,x,ancilla):
        fx = self.applyFx(x)
        newAncilla = str((((int)(ancilla) + fx)%2))
        result = x+newAncilla
        return getQubitVector(result)
    
    '''
    Using pointer to f(x) and n denoting the no. of qubits,
    it returns an oracle matrix Uf of size 2^(n+1)x2^(n+1) which
    can be used in a Deutsch-Jozsa or Bernstein-Vazirani circuit
    '''
    def __createUf(self):
        Uf = np.zeros((2**(self.__n+1),2**(self.__n+1)))    
    
        #this dictionary represents the correspondence between bit combinations and Uf indices
        indices_dict = {}
        counter = 0

        inputs = getAllPossibleNBitStrings(self.__n+1)

        for i in inputs:
            #input to the function is the first n bits of the elements (bit patterns) from the dictionary
            x = i[0:self.__n]

            #fx represents the output of function f given the input x
            fx = str(self.applyFx(x))

            #b is the last bit of the bit pattern in the dictionary item
            b = i[self.__n]

            #below we have the (f(x) + b) mod 2
            if(b==fx):
                bfx = '0'
            else:
                bfx = '1'
                # print(bfx)

            #the final bit string is the concatenation of the input x and bfx
            result = x + bfx

            #using indices_dict we can now find the index that corresponds to this output
            column = getDecimalNo(result)
            row = getDecimalNo(i)
            #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
            Uf[row][column] = 1        
        return Uf
    
    '''
    Given a function and n qubits. This function will verify if the Uf matrix 
    generated for this function matches the expected output. 
    i.e., it checks if Uf|x1x2..xn>|b> = |x1x2..xn> |b+f(x)>
    '''
    def verifyUf(self,debug=False):
        
        UfMatrix = self.getUf()
        if debug:
            print("\nUfMatrix for function \n"+str(UfMatrix)+"\n")

        inputs = getAllPossibleNBitStrings(self.__n+1)
        valid = True

        for i in inputs:
            inputBitVector = getQubitVector(i)

            if debug:
                print("Input BitString {}".format(i))
                print("Input BitVector {}\n".format(inputBitVector))

            functionOutput = self.getFunctionResult(i[0:self.__n],i[self.__n])
            UfOutput = np.matmul(UfMatrix,inputBitVector)

            if debug:
                print("\tUf Output {}".format(UfOutput))
                print("\t F Output {}\n".format(functionOutput))

            if np.array_equal(UfOutput,functionOutput) is not True:
                print("ERROR FOR {}\n".format(i))
                valid=False

        return valid

## Deutsch Jozsa Function Object
A subclass of the function object, this creates a function object for the Deutsch-Jozssa Algorithm.

In [3]:
import random

class DeutschJozsaFunction(FunctionObject):
    
    '''
    Initializes a function object with the passed in fx and ftype values.
    If no fx and ftype values are passed, 
    it will create a balanced or constant function at random and initialize it
    '''
    def __init__(self,n, fx=None,ftype=None):
        if fx is None:
            functionObj = self.createRandomDeutschJozsaFunction(n)
            fx = functionObj.fx
            self.ftype = functionObj.getFunctionType()
        else:
            self.ftype = ftype
        FunctionObject.__init__(self, fx, n)        
        
    '''
    Randomly create a balanced or constant function
    '''    
    def createRandomDeutschJozsaFunction(self,n):
        fTypeOrdinal = random.randint(0,1)
        if fTypeOrdinal == 0:
            return BalancedFunction(n)
        else:
            return ConstantFunction(n)
        
    '''
    CONSTANT or BALANCED based on function type 
    '''
    def getType(self):
        return self.ftype
    

class BalancedFunction(DeutschJozsaFunction):
    
    def __init__(self,n):
        # a random bit chosen from 0 to n-1 which will be 
        # 0 for half inputs and 1 for other half inputs
        self.decisionBit = random.randint(0,n-1)
        
    def fx(self,inputString):
        if inputString[self.decisionBit] == '1':
            return 1
        else:
            return 0
    
    def getFunctionType(self):
        return "BALANCED"
    
class ConstantFunction(DeutschJozsaFunction):
    def __init__(self,n):
        # a random int 0 or 1, which will be the output of this constant function
        self.result = random.randint(0,1)
        
    def fx(self,inputString):
        return self.result
    
    def getFunctionType(self):
        return "CONSTANT"

## Quantum Circuit
The common quantum circuit used for both Deutsch-Jozsa and Berstein-Vazirani Algorithm.

<img src="circuit.png" alt="Circuit" style="width: 300px; float: left"/>

In [4]:
from pyquil import Program
from pyquil.gates import *
from pyquil import get_qc
from pyquil.quilatom import unpack_qubit
from pyquil.quil import DefGate
    
def runMainCircuit(functionObj,nTrials,debug=False):
    p = Program()

    #create a matrix representing Uf
    UfMatrix = functionObj.getUf()
    
    #for a n bit function, we need n+1 qubits(one ancilla bit)
    n = functionObj.getN()+1
    
    '''qc_name = "{}q-qvm".format(n)
    qc = get_qc(qc_name)'''
    
    '''*************************************CHANGE START******************************************'''
    lattice = 'Aspen-4-7Q-A'  # edit as necessary
    qpu = get_qc(lattice)
    #qpu.compiler.timeout = 600
    qubits = qpu.device.qubits()
    print(f'All qubits in this device in order: {qubits}')
    
    reqd_qubits=[]
    
    # includes helper bit due to n+1
    
    for i in range(0,n):
         
        reqd_qubits.append(qubits[i])
    '''**************************************CHANGE OVER******************************************'''
    
    
    #setting last qubit to 1
    p+=X(reqd_qubits[-1])
    
    #adding Hadamard gates to all qubits
    for i in reqd_qubits:
        p+=H(i)

    GateName = "UF_GATE"

    #create a gate that uses the Uf matrix and pass all qubits to this as input 
    uf_gate_definition = DefGate(GateName, UfMatrix)
    qubits_for_gate = [unpack_qubit(i) for i in reqd_qubits]

    #adding Uf gate
    p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits_for_gate))

    for i in reqd_qubits[:-1]:
        p += H(i)

    if debug:
        print(p)
        
    try:
        results = qpu.run_and_measure(p, trials=nTrials)
    except TimeoutError:        
        print("Timeout occured for n: {}".format(n-1))
        return []
        
    return results

## Post-Process Quantum Circuit Results

Process results of measurement of the above defined quantum circuit to gain information

* Function Type in Deutsch-Jozsa
* a and b values in Bernstein-Vazirani

In [5]:
import time

nTrials = 1

'''
Given a Deutsch-Jozsa functionObject
this function returns if its balanced or constant
'''
def isBalancedOrConstant(functionObj):
    start = time.time()
    results = runMainCircuit(functionObj,nTrials)
    end = time.time()
    constant = True
    
    if len(results) == 0:
        return None,None
    print("Key results ", results)
    #for i in range(0,functionObj.getN()):
    
    keys= list(results.keys())[0:functionObj.getN()]
    for i in keys[:-1]:
        if results[i][0] != 0:
            constant = False
            break
    
    timetaken = end-start
    
    if constant:
        return "CONSTANT",timetaken
    else:
        return "BALANCED",timetaken

## Experiments with Quantum Circuits

In [6]:
'''
Driver that verifies the validity of the quantum circuits for different F configurations.
Here F is either constant or balanced
'''
class DeutschJozsaDriver():
    def __init__(self,n,fx,fType):
        self.functionObj = DeutschJozsaFunction(n,fx,fType)
        
    def runAndVerifyQuantumCircuit(self,debug=False):
        print("*"*20)
        if debug:
            print("Verified that the created Uf matrix is valid: {}".format(balancedFxN2Object.verifyUf()))

        fType,circuitRunTime = isBalancedOrConstant(self.functionObj)
        if fType is None:
            return
        print(self.functionObj.getType())
        print(fType)
        assert fType == self.functionObj.getType()
        print("N : {} , FnType: {}, Time : {}".format(self.functionObj.getN(), self.functionObj.getType(), circuitRunTime))
        print("*"*20)

In [7]:
decisionBit = 0

def constantFunction(x):
    return 1

def balancedFunction(x):
    if x[decisionBit] == '1':
        return 1
    else:
        return 0

## Simulation 1

In [11]:
DeutschJozsaDriver(1,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(2,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(3,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(4,balancedFunction,'BALANCED').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(5,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()
DeutschJozsaDriver(6,constantFunction,'CONSTANT').runAndVerifyQuantumCircuit()

********************
All qubits in this device in order: [0, 1, 2, 7, 13, 14, 15]
Key results  {0: array([0]), 1: array([0]), 2: array([0]), 7: array([0]), 13: array([0]), 14: array([0]), 15: array([0])}
CONSTANT
CONSTANT
N : 1 , FnType: CONSTANT, Time : 0.4192051887512207
********************
********************
All qubits in this device in order: [0, 1, 2, 7, 13, 14, 15]
Key results  {0: array([0]), 1: array([0]), 2: array([1]), 7: array([0]), 13: array([0]), 14: array([0]), 15: array([0])}
CONSTANT
CONSTANT
N : 2 , FnType: CONSTANT, Time : 0.7791440486907959
********************
********************
All qubits in this device in order: [0, 1, 2, 7, 13, 14, 15]
Key results  {0: array([1]), 1: array([0]), 2: array([0]), 7: array([1]), 13: array([0]), 14: array([0]), 15: array([0])}
BALANCED
BALANCED
N : 3 , FnType: BALANCED, Time : 1.2245700359344482
********************
********************
All qubits in this device in order: [0, 1, 2, 7, 13, 14, 15]
Key results  {0: array([1]), 1: a